# [Metaprogramming](https://docs.julialang.org/en/latest/manual/metaprogramming/#)

A "meta" program is a program that manipulates programs.

Most common use refers to a program that generates another program.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code, that is to **"treat code as data"** and manipulate it as just another object in Julia.

## Symbols and Expressions

In [ ]:
:a

In [ ]:
typeof(:a)

In [ ]:
eval(:a)

In [ ]:
a = 2

In [ ]:
eval(:a)

In [ ]:
typeof(:+)

In [ ]:
typeof(:sin)

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [ ]:
ex = :(a + b)  # the expression 'a + b'

In [ ]:
typeof(ex)

In [ ]:
b = 7
eval(ex)

An expression is just a Julia object, so we can introspect (find out information about it):

In [ ]:
dump(ex)

In [ ]:
dump(:(x = 3))

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [ ]:
Meta.parse("a + b")

More complicated expressions are represented as **"abstract syntax trees" (ASTs)**, consisting of expressions nested inside expressions:

In [ ]:
ex = :( sin(3a + 2b^2) )

In [ ]:
dump(ex)

We can also manipulate expressions

In [ ]:
blk = quote # multi-line expression
    println("Hello")
end

In [ ]:
eval(blk)

In [ ]:
dump(blk)

In [ ]:
push!(blk.args, :(println("AFTER")))

In [ ]:
blk

In [ ]:
eval(blk)

In [ ]:
x = 2

In [ ]:
:(x + $x)

In [ ]:
x = :y

In [ ]:
:(x + $x)

### Use case: Programmatically define methods

In [ ]:
struct MyNumber
    x::Float64
end

for op = (:sin, :cos, :tan, :log, :exp)
    eval(quote
        Base.$op(a::MyNumber) = MyNumber($op(a.x))
    end)
end

In [ ]:
sin(MyNumber(pi/2))

# Macros

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

Macros are useful in several cases:

to provide a specific notation different than what can normally be written in the language
e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
* to rearrange or delay evaluation of code
* to eliminate boilerplate (repetitive) code
* to automatically generate complex code that would be painful by hand
* to unroll loops for efficiency
* to inline code for efficiency

Macros are invoked using the @ sign, e.g.

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [ ]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

In [ ]:
@twice println("test")

In [ ]:
@macroexpand @twice println("test")

In [ ]:
@macroexpand @time 2 + 2

### [Macro Hygiene](https://docs.julialang.org/en/latest/manual/metaprogramming/#Hygiene-1)

In [ ]:
macro set_z(val)
    :(z = $val)
end

In [ ]:
@set_z(3.0)

In [ ]:
z

In [ ]:
@macroexpand @set_z(3.0)

In [ ]:
macro set_z(val)
    :($(esc(:z)) = $val)
end

In [ ]:
@set_z(3.0)

In [ ]:
z

In [ ]:
@macroexpand @set_z(3.0)

In [ ]:
z

# Custom string literals

In [ ]:
struct KeepZerosFloat{T<:AbstractFloat}
    x::T
    n::Int64 # extra tail of n zeros
end

In [ ]:
KeepZerosFloat(z::T) where {T<:AbstractFloat} = KeepZerosFloat{T}(z, 0)

In [ ]:
KeepZerosFloat(3.0)

In [ ]:
Base.show(io::IO, z::KeepZerosFloat) = print(io, string(z.x) * join(fill('0', z.n)))

In [ ]:
KeepZerosFloat(3.0)

In [ ]:
KeepZerosFloat(3.0000)

In [ ]:
macro k_str(x)
    xF64 = parse(Float64, x)
    s = string(xF64)
    zeros_str = replace(x, s => "")
    @assert zeros_str === join(fill('0', length(zeros_str)))
    
    KeepZerosFloat{Float64}(xF64, length(zeros_str))
end

In [ ]:
k"3.0"

In [ ]:
k"3.000"

In [ ]:
x = k"3.1410000000"

In [ ]:
typeof(x)

# Unrolling

There are many interesting examples of macros in Base. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$
may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a for loop. But if we know the polynomial at **compile time**, this loop may be unrolled using metaprogramming. This is implemented in the Math module in math.jl in Base, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [ ]:
function horner_loop(x, a...)
    r = a[end]
    for i in length(a)-1:-1:1
        r = a[i] + x*r
    end
    r
end

In [ ]:
x = 3
horner_loop(x, 2, 3, 4, 5)

### Macro version

In [ ]:
# Modified from base/math.jl
macro horner(x, a...)
    ex = esc(a[end])
    t = esc(x)
    for i in length(a)-1:-1:1
        ex = :( $(esc(a[i])) + $t * $ex )
    end
    ex
end

In [ ]:
x = 3
@horner(x, 2, 3, 4, 5)

In [ ]:
@macroexpand @horner(x, 2, 3, 4, 5)

In [ ]:
horner(x) = @horner(x, 2, 3, 4, 5)

### Comparison

In [ ]:
@code_warntype horner(x)

In [ ]:
@code_warntype horner_loop(x)

## Unroll.jl

In [ ]:
using Unroll, BenchmarkTools

In [ ]:
function f()
    z = 1.0
    for i in 1:10
        z += i*rand()
    end
    z
end

In [ ]:
@time f()

In [ ]:
@code_lowered f()

In [ ]:
@btime f()

In [ ]:
function g()
    z = 1.0
    @unroll for i in 1:10
        z += i*rand()
    end 
    z
end

In [ ]:
@time g()

In [ ]:
@btime g();

In [ ]:
@code_lowered g()

Based on https://github.com/mitmath/18S096/blob/master/lectures/lecture7/Metaprogramming.ipynb